# GSOWeatherParseTest

## Objectives:
### Test the Metar package
### Decode the Metar codes
#### Have dataframe completely decoded for HOURLYSKYCONDITIONS and HOURLYPRESENTWEATHERTTYPE


In [165]:
import numpy as np
import pandas as pd
import re
from metar import Metar

In [166]:
####THIS IS A TEST SAMPLE LOOKING AT WHAT PYTHON_METAR CAN DO####

# A sample METAR report
code = "METAR KEWR 111851Z VRB03G19KT 2SM R04R/3000VP6000FT TSRA BR FEW015 BKN040CB BKN065 OVC200 22/22 A2987 RMK AO2 PK WND 29028/1817 WSHFT 1812 TSB05RAB22 SLP114 FRQ LTGICCCCG TS OHD AND NW-N-E MOV NE P0013 T02270215"

print("-----------------------------------------------------------------------")
print("METAR: ",code)
print("-----------------------------------------------------------------------")

# Initialize a Metar object with the coded report
obs = Metar.Metar(code)

# Print the individual data

# The 'station_id' attribute is a string.
print("station: %s" % obs.station_id)

if obs.type:
  print("type: %s" % obs.report_type())

# The 'time' attribute is a datetime object
if obs.time:
  print("time: %s" % obs.time.ctime())

# The 'temp' and 'dewpt' attributes are temperature objects
if obs.temp:
  print("temperature: %s" % obs.temp.string("C"))

if obs.dewpt:
  print("dew point: %s" % obs.dewpt.string("C"))

# The wind() method returns a string describing wind observations
# which may include speed, direction, variability and gusts.
if obs.wind_speed:
  print("wind: %s" % obs.wind())

# The peak_wind() method returns a string describing the peak wind 
# speed and direction.
if obs.wind_speed_peak:
  print("wind: %s" % obs.peak_wind())

# The visibility() method summarizes the visibility observation.
if obs.vis:
  print("visibility: %s" % obs.visibility())

# The runway_visual_range() method summarizes the runway visibility
# observations.
if obs.runway:
  print("visual range: %s" % obs.runway_visual_range())

# The 'press' attribute is a pressure object.
if obs.press:
  print("pressure: %s" % obs.press.string("mb"))

# The 'precip_1hr' attribute is a precipitation object.
if obs.precip_1hr:
  print("precipitation: %s" % obs.precip_1hr.string("in"))

# The present_weather() method summarizes the weather description (rain, etc.)
print("weather: %s" % obs.present_weather())

# The sky_conditions() method summarizes the cloud-cover observations.
print("sky: %s" % obs.sky_conditions("\n     "))

# The remarks() method describes the remark groups that were parsed, but 
# are not available directly as Metar attributes.  The precipitation, 
# min/max temperature and peak wind remarks, for instance, are stored as
# attributes and won't be listed here.
if obs._remarks:
  print("remarks:")
  print("- "+obs.remarks("\n- "))

print("-----------------------------------------------------------------------\n")



-----------------------------------------------------------------------
('METAR: ', 'METAR KEWR 111851Z VRB03G19KT 2SM R04R/3000VP6000FT TSRA BR FEW015 BKN040CB BKN065 OVC200 22/22 A2987 RMK AO2 PK WND 29028/1817 WSHFT 1812 TSB05RAB22 SLP114 FRQ LTGICCCCG TS OHD AND NW-N-E MOV NE P0013 T02270215')
-----------------------------------------------------------------------
station: KEWR
type: routine report, cycle 19 (automatic report)
time: Wed Oct 11 18:51:00 2017
temperature: 22.7 C
dew point: 21.5 C
wind: variable at 3 knots, gusting to 19 knots
wind: WNW at 28 knots at 18:17
visibility: 2 miles
visual range: on runway 04R, from 3000 to greater than 6000 meters
pressure: 1011.5 mb
precipitation: 0.13in
weather: thunderstorm with rain; mist
sky: a few clouds at 1500 feet
     broken cumulonimbus at 4000 feet
     broken clouds at 6500 feet
     overcast at 20000 feet
remarks:
- Automated station (type 2)
- peak wind 28kt from 290 degrees at 18:17
- wind shift at 18:12
- frequent lightnin

In [167]:
df=pd.read_csv(r'../data/1052640.csv')

In [168]:
df.rename(columns = {'REPORTTPYE':'REPORTTYPE'}, inplace=True)

In [169]:
df = df[df.REPORTTYPE != 'SOD']
df = df[df.REPORTTYPE != 'FM-12']
df.REPORTTYPE.unique()

array(['FM-15', 'FM-16', 'SY-MT'], dtype=object)

In [170]:
def skyConditionsFormat(x):
    splitting_x = re.split('(\W+)', x)
    for i in splitting_x:
        if i == ' ':
            pass
        elif len(i) < 5:
            hash2 = i[:3] + '0' + i[3:]
            splitting_x = [j.replace(i, hash2) for j in splitting_x]
        else:
            pass 
    return ''.join(splitting_x)

In [171]:
def getSkyConditions(x):
    if x in ('CLR:00', 'CLR00:000000', 'CLR0:0000'):
        return 'Clear sky'
    elif x in ('X00 SCT70', '0'):
        pass
    elif "s" in x:
        pass
    else:
        obs = Metar.Metar(x)
        return obs.sky_conditions()

In [172]:
def getWeatherConditions(y):
    if y in ('0'):
        pass
    elif "s" in y:
        pass
    else:
        obs = Metar.Metar(y)
        return obs.present_weather()

### SKYCONDITIONS translates the codes properly
##### Had to remove the 2 integers before the Sky code EX) FEW:01 XXX; the "01" is the layer amount used in conjunction with the sky condition code given in eighths (oktas) so 01-02 correspond to the code FEW so I removed these integers since they correspond to the code it follows and so that the metar package would also work in translating 
###### I had to also remove some data that had the "s" ("Suspect value" that Brown mentioned) within the data, these values continued to give errors so I had them skipped during translation
###### Also at the moment I have the 'NaN' values replaced with '' but I can replace if needed

#### Remove chars : _ and 2 integers following SKYCONDITIONS code; looking at LCD shows these 2 digits correspond to the SKYCONDITIONS code it follows

In [173]:
df['HOURLYSKYCONDITIONS'] = df['HOURLYSKYCONDITIONS'].str.replace(r'\:\d{2}\s', '')

In [174]:
df['HOURLYSKYCONDITIONS'] = df['HOURLYSKYCONDITIONS'].replace(np.nan, '', regex=True)

### 3 Skyconditions values with the "s"  

In [175]:
df[df['HOURLYSKYCONDITIONS'].str.contains("s")]

,STATION,STATION_NAME,ELEVATION,LATITUDE,LONGITUDE,DATE,REPORTTYPE,HOURLYSKYCONDITIONS,HOURLYVISIBILITY,HOURLYPRSENTWEATHERTYPE,...,MonthlyMaxSeaLevelPressureTime,MonthlyMinSeaLevelPressureValue,MonthlyMinSeaLevelPressureDate,MonthlyMinSeaLevelPressureTime,MonthlyTotalHeatingDegreeDays,MonthlyTotalCoolingDegreeDays,MonthlyDeptFromNormalHeatingDD,MonthlyDeptFromNormalCoolingDD,MonthlyTotalSeasonToDateHeatingDD,MonthlyTotalSeasonToDateCoolingDD
7153,WBAN:13723,GREENSBORO AIRPORT NC US,271.3,36.0969,-79.9432,2017-07-05 18:54,FM-15,FEW:02s 15s SCT:04s 40s,10.00,|TS:91 |,...,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN
7619,WBAN:13723,GREENSBORO AIRPORT NC US,271.3,36.0969,-79.9432,2017-07-18 17:04,FM-16,FEW:02s 20s BKN:07s 35s,10.00,|TS:91 |,...,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN
8528,WBAN:13723,GREENSBORO AIRPORT NC US,271.3,36.0969,-79.9432,2017-08-12 11:54,FM-15,SCT:04s 24s BKN:07s 30s,10.00,NaN,...,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN


#### Translate code

In [176]:
df['HOURLYSKYCONDITIONS'] = df['HOURLYSKYCONDITIONS'].apply(skyConditionsFormat)
df['HOURLYSKYCONDITIONS'] = df['HOURLYSKYCONDITIONS'].apply(getSkyConditions)
df

,STATION,STATION_NAME,ELEVATION,LATITUDE,LONGITUDE,DATE,REPORTTYPE,HOURLYSKYCONDITIONS,HOURLYVISIBILITY,HOURLYPRSENTWEATHERTYPE,...,MonthlyMaxSeaLevelPressureTime,MonthlyMinSeaLevelPressureValue,MonthlyMinSeaLevelPressureDate,MonthlyMinSeaLevelPressureTime,MonthlyTotalHeatingDegreeDays,MonthlyTotalCoolingDegreeDays,MonthlyDeptFromNormalHeatingDD,MonthlyDeptFromNormalCoolingDD,MonthlyTotalSeasonToDateHeatingDD,MonthlyTotalSeasonToDateCoolingDD
0,WBAN:13723,GREENSBORO AIRPORT NC US,271.3,36.0969,-79.9432,2017-01-01 00:54,FM-15,a few clouds at 5000 feet; broken clouds at 75...,10.00,NaN,...,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN
1,WBAN:13723,GREENSBORO AIRPORT NC US,271.3,36.0969,-79.9432,2017-01-01 01:54,FM-15,broken clouds at 6500 feet; broken clouds at 7...,10.00,NaN,...,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN
2,WBAN:13723,GREENSBORO AIRPORT NC US,271.3,36.0969,-79.9432,2017-01-01 02:54,FM-15,broken clouds at 6500 feet; broken clouds at 9...,10.00,NaN,...,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN
3,WBAN:13723,GREENSBORO AIRPORT NC US,271.3,36.0969,-79.9432,2017-01-01 03:54,FM-15,overcast at 7000 feet,10.00,NaN,...,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN
4,WBAN:13723,GREENSBORO AIRPORT NC US,271.3,36.0969,-79.9432,2017-01-01 04:54,FM-15,a few clouds at 1300 feet; scattered clouds at...,7.00,-RA:02 |RA:61 |,...,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN
5,WBAN:13723,GREENSBORO AIRPORT NC US,271.3,36.0969,-79.9432,2017-01-01 05:54,FM-15,a few clouds at 1300 feet; scattered clouds at...,9.00,NaN,...,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN
6,WBAN:13723,GREENSBORO AIRPORT NC US,271.3,36.0969,-79.9432,2017-01-01 06:52,FM-16,a few clouds at 900 feet; broken clouds at 600...,10.00,NaN,...,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN
7,WBAN:13723,GREENSBORO AIRPORT NC US,271.3,36.0969,-79.9432,2017-01-01 06:54,FM-15,a few clouds at 900 feet; broken clouds at 600...,10.00,NaN,...,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN
8,WBAN:13723,GREENSBORO AIRPORT NC US,271.3,36.0969,-79.9432,2017-01-01 07:13,FM-16,broken clouds at 2000 feet; broken clouds at 6...,10.00,NaN,...,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN
9,WBAN:13723,GREENSBORO AIRPORT NC US,271.3,36.0969,-79.9432,2017-01-01 07:52,FM-16,a few clouds at 700 feet; scattered clouds at ...,10.00,NaN,...,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN


### HOURLYPRESENTWEATHERTYPE translates the codes properly
###### I had to also remove some data that had the "s" ("Suspect value" that Brown mentioned) within the data, these values continued to give errors so I had them skipped during translation
###### Also at the moment I have the 'NaN' values replaced with '' but I can replace if needed

In [177]:
df['HOURLYPRSENTWEATHERTYPE'] = df['HOURLYPRSENTWEATHERTYPE'].str.replace(r'\:\d*\s', ' ')
df['HOURLYPRSENTWEATHERTYPE'] = df['HOURLYPRSENTWEATHERTYPE'].str.replace(r'\:\d*', ' ')
df['HOURLYPRSENTWEATHERTYPE'] = df['HOURLYPRSENTWEATHERTYPE'].str.replace(r'\|', '')
df['HOURLYPRSENTWEATHERTYPE'] = df['HOURLYPRSENTWEATHERTYPE'].str.rstrip()

In [178]:
df['HOURLYPRSENTWEATHERTYPE'] = df['HOURLYPRSENTWEATHERTYPE'].replace(np.nan, '', regex=True)

### 29 HourlyWeatherType values with the "s"  

In [179]:
df[df['HOURLYPRSENTWEATHERTYPE'].str.contains("s")]

,STATION,STATION_NAME,ELEVATION,LATITUDE,LONGITUDE,DATE,REPORTTYPE,HOURLYSKYCONDITIONS,HOURLYVISIBILITY,HOURLYPRSENTWEATHERTYPE,...,MonthlyMaxSeaLevelPressureTime,MonthlyMinSeaLevelPressureValue,MonthlyMinSeaLevelPressureDate,MonthlyMinSeaLevelPressureTime,MonthlyTotalHeatingDegreeDays,MonthlyTotalCoolingDegreeDays,MonthlyDeptFromNormalHeatingDD,MonthlyDeptFromNormalCoolingDD,MonthlyTotalSeasonToDateHeatingDD,MonthlyTotalSeasonToDateCoolingDD
271,WBAN:13723,GREENSBORO AIRPORT NC US,271.3,36.0969,-79.9432,2017-01-06 23:19,FM-16,,0.50,-SN BR s SN s,...,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN
280,WBAN:13723,GREENSBORO AIRPORT NC US,271.3,36.0969,-79.9432,2017-01-07 03:28,FM-16,scattered clouds at 700 feet; overcast at 1300...,0.50,-SN BR s SN s,...,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN
281,WBAN:13723,GREENSBORO AIRPORT NC US,271.3,36.0969,-79.9432,2017-01-07 03:54,FM-15,scattered clouds at 700 feet; overcast at 1400...,0.50,-SN BR s SN s,...,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN
825,WBAN:13723,GREENSBORO AIRPORT NC US,271.3,36.0969,-79.9432,2017-01-21 08:23,FM-16,overcast at 200 feet,0.50,BR s,...,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN
826,WBAN:13723,GREENSBORO AIRPORT NC US,271.3,36.0969,-79.9432,2017-01-21 08:54,FM-15,overcast at 200 feet,0.50,BR s,...,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN
882,WBAN:13723,GREENSBORO AIRPORT NC US,271.3,36.0969,-79.9432,2017-01-22 07:29,FM-16,overcast at 200 feet,0.50,BR s,...,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN
890,WBAN:13723,GREENSBORO AIRPORT NC US,271.3,36.0969,-79.9432,2017-01-22 09:41,FM-16,,0.25,BR s,...,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN
908,WBAN:13723,GREENSBORO AIRPORT NC US,271.3,36.0969,-79.9432,2017-01-22 17:54,FM-15,overcast at 200 feet,0.25,-DZ BR s DZ s,...,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN
912,WBAN:13723,GREENSBORO AIRPORT NC US,271.3,36.0969,-79.9432,2017-01-22 19:32,FM-16,overcast at 200 feet,0.50,-DZ BR s DZ s,...,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN
913,WBAN:13723,GREENSBORO AIRPORT NC US,271.3,36.0969,-79.9432,2017-01-22 19:54,FM-15,overcast at 200 feet,0.50,-RA BR s RA s,...,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN


###### Translate codes

In [180]:
df['HOURLYPRSENTWEATHERTYPE'] = df['HOURLYPRSENTWEATHERTYPE'].apply(getWeatherConditions)
df

,STATION,STATION_NAME,ELEVATION,LATITUDE,LONGITUDE,DATE,REPORTTYPE,HOURLYSKYCONDITIONS,HOURLYVISIBILITY,HOURLYPRSENTWEATHERTYPE,...,MonthlyMaxSeaLevelPressureTime,MonthlyMinSeaLevelPressureValue,MonthlyMinSeaLevelPressureDate,MonthlyMinSeaLevelPressureTime,MonthlyTotalHeatingDegreeDays,MonthlyTotalCoolingDegreeDays,MonthlyDeptFromNormalHeatingDD,MonthlyDeptFromNormalCoolingDD,MonthlyTotalSeasonToDateHeatingDD,MonthlyTotalSeasonToDateCoolingDD
0,WBAN:13723,GREENSBORO AIRPORT NC US,271.3,36.0969,-79.9432,2017-01-01 00:54,FM-15,a few clouds at 5000 feet; broken clouds at 75...,10.00,None,...,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN
1,WBAN:13723,GREENSBORO AIRPORT NC US,271.3,36.0969,-79.9432,2017-01-01 01:54,FM-15,broken clouds at 6500 feet; broken clouds at 7...,10.00,None,...,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN
2,WBAN:13723,GREENSBORO AIRPORT NC US,271.3,36.0969,-79.9432,2017-01-01 02:54,FM-15,broken clouds at 6500 feet; broken clouds at 9...,10.00,None,...,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN
3,WBAN:13723,GREENSBORO AIRPORT NC US,271.3,36.0969,-79.9432,2017-01-01 03:54,FM-15,overcast at 7000 feet,10.00,None,...,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN
4,WBAN:13723,GREENSBORO AIRPORT NC US,271.3,36.0969,-79.9432,2017-01-01 04:54,FM-15,a few clouds at 1300 feet; scattered clouds at...,7.00,light rain; rain,...,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN
5,WBAN:13723,GREENSBORO AIRPORT NC US,271.3,36.0969,-79.9432,2017-01-01 05:54,FM-15,a few clouds at 1300 feet; scattered clouds at...,9.00,None,...,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN
6,WBAN:13723,GREENSBORO AIRPORT NC US,271.3,36.0969,-79.9432,2017-01-01 06:52,FM-16,a few clouds at 900 feet; broken clouds at 600...,10.00,None,...,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN
7,WBAN:13723,GREENSBORO AIRPORT NC US,271.3,36.0969,-79.9432,2017-01-01 06:54,FM-15,a few clouds at 900 feet; broken clouds at 600...,10.00,None,...,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN
8,WBAN:13723,GREENSBORO AIRPORT NC US,271.3,36.0969,-79.9432,2017-01-01 07:13,FM-16,broken clouds at 2000 feet; broken clouds at 6...,10.00,None,...,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN
9,WBAN:13723,GREENSBORO AIRPORT NC US,271.3,36.0969,-79.9432,2017-01-01 07:52,FM-16,a few clouds at 700 feet; scattered clouds at ...,10.00,None,...,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN
